In [5]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [6]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [7]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'title': '東京駅(東京都)周辺のイベント - ウォーカープラス',
   'content': '開催中 2025年12月20日(土)～2026年2月22日(日). 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都). CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ). 日比谷駅(東京都), 有楽町駅(東京都), 東京駅(東京都), 京橋駅(東京都). 開催中 2026年1月2日(金)～3月22日(日). 二重橋前駅(東京都), 有楽町駅(東京都), 東京駅(東京都), 日比谷駅(東京都), 銀座一丁目駅(東京都). * 歴史リアル謎解きゲーム「謎の城」in 日本橋「発明家／人斬り-平賀源内-」. 開催中 2025年9月1日(月)～2026年3月1日(日). 日本橋駅(東京都), 京橋駅(東京都), 東京駅(東京都), 宝町駅(東京都), 三越前駅(東京都). * MIDTOWN YAESU CHRISTMAS 2025 (ミッドタウン八重洲クリスマス2025). 終了間近 2025年11月13日(木)～2026年2月15日(日). 京橋駅(東京都), 東京駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都). * 丸の内から御食国「敦賀・若狭」へ 美し物-うましもの-グルメフェア. 開催中 2026年2月2日(月)～21日(土). 二重橋前駅(東京都), 有楽町駅(東京都), 東京駅(東京都), 日比谷駅(東京都). 終了間近 2025年11月13日(木)～2026年2月15日(日). 二重橋前〈丸の内〉駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 日比谷駅(東京都), 銀座一丁目駅(東京都). * Masking Tape Jamboree in KITTE2026. 終了間近 2026年2月11日(水)～13日(金). 東京駅(東京都), 二重橋前駅(東京都), 有楽町駅(東京都), 京橋駅(東京都), 銀座一丁目駅(東京都). 東京駅(東京都), 二重橋前駅

In [8]:
# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [9]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [10]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [11]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [12]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

- 東京都の面積は約2,194平方キロメートルです。
- 沖縄県の面積は約2,271平方キロメートルです。

したがって、沖縄県が東京都よりも広いということになります。


In [15]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
最近1ヶ月以内に東京駅で開催されるイベントについての情報をいくつかご紹介します。

1. **東京駅のイベント情報一覧**
   - **リンク**: [東京駅のイベント情報](https://ekitan.com/event/station-2590)
   - 東京駅周辺でのイベントが多数掲載されています。

2. **グランスタ東京でのポップアップショップ**
   - **リンク**: [Instagram Post](https://www.instagram.com/p/DQdkHJHEuqj/)
   - JR東京駅構内の「グランスタ東京」にて、1ヶ月間のポップアップショップが開催されます。

3. **東京イベント情報**
   - **リンク**: [My TOKYOのイベント情報](https://www.my.metro.tokyo.lg.jp/event)
   - 東京都内での様々なイベント情報を提供しています。

4. **東京のおすすめイベント**
   - **リンク**: [Enjoy Tokyoのイベント情報](https://www.enjoytokyo.jp/event/list/)
   - 東京全域の最新のイベント情報を確認できます。

5. **日程に基づいたイベント検索**
   - **リンク**: [イベント情報の詳細](https://smout.jp/plans/20620)
   - 特定の日程で開催するイベントの情報を集めています。

これらのリンクを参考にして、興味のあるイベントを見つけてみてください。また、イベントの詳細や開催場所、参加方法などは各リンク先で確認できます。


In [16]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:東北6県は？'

東北地方は、日本の北部に位置し、以下の6つの県から構成されています。

1. 青森県 (あおもりけん)
2. 岩手県 (いわてけん)
3. 宮城県 (みやぎけん)
4. 秋田県 (あきたけん)
5. 山形県 (やまがたけん)
6. 福島県 (ふくしまけん) 

これらの県は、自然や文化、歴史が豊かで、多様な観光地があります。


'質問:宮城県のお土産について検索した結果を教えて'

宮城県のお土産には、特に人気のあるアイテムがいくつかあります。以下は、その一部です：

1. **牛たん** - 宮城県は牛たん発祥の地として知られ、厚切りの牛たん焼きは特に人気です。
2. **笹かまぼこ** - 新鮮な魚のすり身で作られた、笹の葉に包まれた形状のかまぼこです。
3. **ずんだ餅** - 青えんどう豆をつぶして作るずんだを使ったお菓子で、甘さが絶妙です。
4. **仙台味噌** - 地元の食材を使った、豊かな風味の味噌です。
5. **萩の月** - カステラ生地にカスタードクリームが入ったお菓子で、定番の土産として人気があります。

さらに、オンラインでのお土産購入も可能で、地域の特産品を取り揃えたサイトもあります。特に、地方の名品や独自のスイーツ、伝統的な加工品など、多様な選択肢があるため、お好みや贈る相手に応じて選ぶことができます。

詳しい情報は、以下のリンクを参照してください：
- [宮城県の名物お土産について](https://shunsentanbou.pref.miyagi.jp/theme/miyage/)
- [観光地でのお土産ランキング](https://www.jalan.net/news/article/93858/) 

このように宮城県には、訪れる人々にとって非常に魅力的なお土産が揃っています。

---ご利用ありがとうございました！---
